In [ ]:
#refer week 5 video

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
from datetime import datetime
import os, sys

HOMEPATH = "/home/ubuntu/fastai/"

os.chdir(HOMEPATH)

In [3]:
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using TensorFlow backend.
Using Theano backend.


In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

## Setup

In [5]:
path = HOMEPATH + "data/dogscats/"
model_path = path + 'models/'
print (path)
print (model_path)

/home/ubuntu/fastai/data/dogscats/
/home/ubuntu/fastai/data/dogscats/models/


In [6]:

if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64

In [7]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [8]:
print (type(batches))
print (type(val_batches))

<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'keras.preprocessing.image.DirectoryIterator'>


In [9]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [10]:
print ("val_classes:", type(val_classes), val_classes.shape)
print ("trn_classes:", type(trn_classes), trn_classes.shape)
print ("val_labels:", type(val_labels), val_labels.shape)
print ("trn_labels:", type(trn_labels), trn_labels.shape)
print ("val_filenames:", type(val_filenames), len(val_filenames))
print ("filenames:", type(filenames), len(filenames))
print ("test_filenames:", type(test_filenames), len(test_filenames))

val_classes: <type 'numpy.ndarray'> (2000,)
trn_classes: <type 'numpy.ndarray'> (23000,)
val_labels: <type 'numpy.ndarray'> (2000, 2)
trn_labels: <type 'numpy.ndarray'> (23000, 2)
val_filenames: <type 'list'> 2000
filenames: <type 'list'> 23000
test_filenames: <type 'list'> 12500


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer  
2) Add this to a model containing all VGG layers except the last layer  
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)  
4) Add data augmentation, fine-tuning the dense layers without pre-computation.  

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [11]:
model = Vgg16().model
conv_layers,fc_layers = split_at(model, Convolution2D)

In [12]:
print ("model:", type(model), type(model.layers), len(model.layers))
print ("conv_layers:", type(conv_layers), len(conv_layers))
print ("fc_layers:", type(fc_layers), len(fc_layers))

for layer in model.layers:
    print (layer)

model: <class 'keras.models.Sequential'> <type 'list'> 38
conv_layers: <type 'list'> 31
fc_layers: <type 'list'> 7


In [13]:
conv_model = Sequential(conv_layers)

In [14]:
print ("conv_model:", type(conv_model))
print ("conv_model:", type(conv_model.layers), len(conv_model.layers))
for layer in model.layers:
    print (layer)

conv_model: <class 'keras.models.Sequential'>
conv_model: <type 'list'> 31


In [15]:
print("start")
startTime= datetime.now()
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


startTime= datetime.now()
trn_features = conv_model.predict_generator(batches, batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

#takes long time.

start
Time elpased (hh:mm:ss.ms) 0:00:50.864010
Time elpased (hh:mm:ss.ms) 0:09:04.765339


In [16]:
print ("val_features:", type(val_features))
print ("trn_features:", type(trn_features))

val_features: <type 'numpy.ndarray'>
trn_features: <type 'numpy.ndarray'>


In [17]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In the future we can just load these precomputed features:

In [18]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [19]:
print ("val_features:", type(val_features), val_features.shape)
print ("trn_features:", type(trn_features), trn_features.shape)

val_features: <type 'numpy.ndarray'> (2000, 512, 14, 14)
trn_features: <type 'numpy.ndarray'> (23000, 512, 14, 14)


We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [20]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [21]:
save_array(model_path+'train_data.bc', trn)
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [22]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

In [23]:
print (type(trn))
print (type(val))

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [24]:
print (type(model), len(model.layers))
for layer in model.layers:
    print (layer)

<class 'keras.models.Sequential'> 38


In [25]:
model.pop()
model.pop()

In [26]:
ll_val_feat = model.predict_generator(val_batches, val_batches.nb_sample)
ll_feat = model.predict_generator(batches, batches.nb_sample)

In [27]:
save_array(model_path + 'train_ll_feat.bc', ll_feat)
save_array(model_path + 'valid_ll_feat.bc', ll_val_feat)

In [28]:
print ("ll_feat:", type(ll_feat))
print ("ll_val_feat:", type(ll_val_feat))



ll_feat: <type 'numpy.ndarray'>
ll_val_feat: <type 'numpy.ndarray'>


In [29]:
ll_feat = load_array(model_path+ 'train_ll_feat.bc')
ll_val_feat = load_array(model_path + 'valid_ll_feat.bc')

...and let's also grab the test data, for when we need to submit:

In [30]:
test = get_data(path+'test')
save_array(model_path+'test_data.bc', test)

Found 12500 images belonging to 1 classes.


In [31]:
test = load_array(model_path+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [32]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [33]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model

## Dense model

In [34]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [35]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [36]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    if int(i) > 0:
        print 'you info'
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, 
       width_zoom_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    for layer in conv_model.layers[16:]: layer.trainable = True
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=8, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=10, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

## Build ensemble

In [37]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    if int(i) > 0:
        model.summary()
    train_dense_layers(i, model)

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 3s - loss: 0.7784 - acc: 0.6992 - val_loss: 2.1623 - val_acc: 0.3815
Epoch 2/12
23000/23000 [==============================] - 3s - loss: 0.4855 - acc: 0.8441 - val_loss: 2.6026 - val_acc: 0.3725
Epoch 3/12
23000/23000 [==============================] - 3s - loss: 0.4525 - acc: 0.8658 - val_loss: 2.7598 - val_acc: 0.3690
Epoch 4/12
23000/23000 [==============================] - 3s - loss: 0.4335 - acc: 0.8743 - val_loss: 2.8023 - val_acc: 0.3650
Epoch 5/12
23000/23000 [==============================] - 3s - loss: 0.4256 - acc: 0.8791 - val_loss: 2.7783 - val_acc: 0.3635
Epoch 6/12
23000/23000 [==============================] - 3s - loss: 0.4174 - acc: 0.8808 - val_loss: 2.7921 - val_acc: 0.3645
Epoch 7/12
23000/23000 [==============================] - 3s - loss: 0.4049 - acc: 0.8857 - val_loss: 2.7961 - val_acc: 0.3650
Epoch 8/12
23000/23000 [==============================] - 3s -

ValueError: You called `set_weights(weights)` on layer "batchnormalization_3" with a  weight list of length 0, but the layer was expecting 4 weights. Provided weights: []...

## Combine ensemble and test

In [ ]:
ens_model = vgg_ft(2)
for layer in ens_model.layers: layer.trainable=True

In [ ]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in range(5):
        i = str(i)
        ens_model.load_weights('{}{}{}.h5'.format(model_path, fname, i))
        preds = ens_model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return ens_pred

In [ ]:
val_pred2 = get_ens_pred(val, 'aug')

In [ ]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [ ]:
categorical_accuracy(val_labels, val_avg_preds2).eval()